In [ ]:
# ## Prerequisite. Run on Terminal. Install miniconda. Navigate to the folder cd /home/asus/

# mkdir -p ~/miniconda3
# wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
# bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
# rm -rf ~/miniconda3/miniconda.sh
# ~/miniconda3/bin/conda init bash
# ~/miniconda3/bin/conda init zsh

In [ ]:
#### RUn on terminal: Create a virtual environment, Install all the requirements like pytorch, xformers, cuda etc.
#### install the unsloth library and other libraries viz. peft

# conda create --name unsloth_env python=3.11 pytorch-cuda=12.1 pytorch cudatoolkit xformers -c pytorch -c nvidia -c xformers -y
# conda activate unsloth_env
# pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# pip install --no-deps trl peft accelerate bitsandbytes

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4060 Laptop GPU. Max memory: 7.996 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [14]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [15]:
alpaca_prompt= """Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.

### sql_prompt:
{}

### sql:
{}

### Explanation:
{}"""

In [16]:
alpaca_prompt

'Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\n{}\n\n### sql:\n{}\n\n### Explanation:\n{}'

In [17]:
def formatting_prompts_func(examples):
    sql_prompt = examples["sql_prompt"]
    sql = examples["sql"]
    sql_explanation = examples['sql_explanation']

    texts = []
    
    EOS_TOKEN = tokenizer.eos_token
    for sql_prompt, sql, sql_explanation in zip(sql_prompt, sql, sql_explanation):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(sql_prompt, sql, sql_explanation) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [18]:
from datasets import load_dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql", split = "train")
print(dataset.column_names)

['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation']


In [19]:
dataset

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 100000
})

In [20]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [21]:
dataset

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation', 'text'],
    num_rows: 100000
})

In [22]:
dataset["text"][0:4]

['Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\nWhat is the total volume of timber sold by each salesperson, sorted by salesperson?\n\n### sql:\nSELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;\n\n### Explanation:\nJoins timber_sales and salesperson tables, groups sales by salesperson, calculates total volume sold by each salesperson, and orders the results by total volume in descending order.<|end_of_text|>',
 'Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\nList all the unique equipment types and their corresponding total maintenance frequency from the equipm

In [23]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 3,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps =  60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [12]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4060 Laptop GPU. Max memory = 7.996 GB.
5.613 GB of memory reserved.


In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


  0%|          | 0/60 [00:00<?, ?it/s]

{'loss': 2.4302, 'grad_norm': 0.8122252821922302, 'learning_rate': 6.666666666666667e-05, 'epoch': 0.0}
{'loss': 2.3642, 'grad_norm': 0.9944256544113159, 'learning_rate': 0.00013333333333333334, 'epoch': 0.0}
{'loss': 2.4076, 'grad_norm': 0.8963799476623535, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 1.9254, 'grad_norm': 1.013152837753296, 'learning_rate': 0.00019649122807017543, 'epoch': 0.0}
{'loss': 1.6032, 'grad_norm': 1.035999059677124, 'learning_rate': 0.00019298245614035088, 'epoch': 0.0}
{'loss': 1.5834, 'grad_norm': 1.3411928415298462, 'learning_rate': 0.00018947368421052632, 'epoch': 0.0}
{'loss': 1.3792, 'grad_norm': 1.6788804531097412, 'learning_rate': 0.00018596491228070177, 'epoch': 0.0}
{'loss': 0.8949, 'grad_norm': 1.4439940452575684, 'learning_rate': 0.0001824561403508772, 'epoch': 0.0}
{'loss': 0.9231, 'grad_norm': 1.535654902458191, 'learning_rate': 0.00017894736842105264, 'epoch': 0.0}
{'loss': 0.9494, 'grad_norm': 1.0220212936401367, 'learning_rate': 0.0001754

In [25]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

88.6071 seconds used for training.
1.48 minutes used for training.
Peak reserved memory = 6.148 GB.
Peak reserved memory for training = 0.535 GB.
Peak reserved memory % of max memory = 76.888 %.
Peak reserved memory for training % of max memory = 6.691 %.


In [ ]:
# model.save_pretrained("lora_model") # Local saving
# tokenizer.save_pretrained("lora_model")

In [16]:
import torch
torch.cuda.empty_cache()


In [17]:
# model.save_pretrained_gguf("model", tokenizer, quantization_method = "f32")

Follow this Steps for Local Saving

In [27]:
model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 0.0 out of 7.57 RAM for saving.


100%|██████████| 32/32 [00:22<00:00,  1.45it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


In [28]:
!git clone --recursive https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 34421, done.
remote: Counting objects: 100% (10690/10690), done.
remote: Compressing objects: 100% (848/848), done.
remote: Total 34421 (delta 10377), reused 9871 (delta 9841), pack-reused 23731 (from 1)
Receiving objects: 100% (34421/34421), 57.71 MiB | 17.94 MiB/s, done.
Resolving deltas: 100% (24998/24998), done.
Submodule 'kompute' (https://github.com/nomic-ai/kompute.git) registered for path 'ggml/src/kompute'
Cloning into '/home/asus/1-Finetune/llama.cpp/ggml/src/kompute'...
remote: Enumerating objects: 9114, done.        
remote: Counting objects: 100% (249/249), done.        
remote: Compressing objects: 100% (143/143), done.        
remote: Total 9114 (delta 116), reused 183 (delta 96), pack-reused 8865 (from 1)        
Receiving objects: 100% (9114/9114), 17.58 MiB | 13.73 MiB/s, done.
Resolving deltas: 100% (5723/5723), done.
Submodule path 'ggml/src/kompute': checked out '4565194ed7c32d1d2efa32ceab4d3c6cae006306'


In [29]:
!make clean -C llama.cpp

make: Entering directory '/home/asus/1-Finetune/llama.cpp'
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE

In [30]:
!make all -j -C llama.cpp

make: Entering directory '/home/asus/1-Finetune/llama.cpp'
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE

In [31]:
!pip install gguf protobuf

In [36]:
!python llama.cpp/convert_hf_to_gguf.py merged_model --outfile text2sql_8.gguf --outtype q8_0

INFO:hf-to-gguf:Loading model: merged_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> Q8_0, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> Q8_0, shape = {14336, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> Q8_0, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> Q8_0, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfloat16 --> Q8_0, shape = {4096, 1024}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.bfloat16 --> Q

INSTALL OLLAMA using Terminal

In [33]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
[sudo] password for asus: 


In [37]:
!ollama create unsloth_model_8 -f Modelfile

transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠦ transferring model data ⠇ transferring model data ⠏ transferring model data ⠏ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠸ transferring model data ⠴ transferring model data ⠦ transferring model data ⠦ transferring model data ⠇ transferring model data ⠇ transferring model data ⠏ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠏ transferring model data ⠏ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠧ transferring

In [20]:
alpaca_prompt1= """Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.

### sql_prompt:
{}

### sql:
{}

### Explanation:
{}"""

In [21]:
input = alpaca_prompt1.format(
    "What is the total volume of timber sold by each salesperson, sorted by salesperson?",
    "",
    "",)

In [22]:
input

'Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\nWhat is the total volume of timber sold by each salesperson, sorted by salesperson?\n\n### sql:\n\n\n### Explanation:\n'

In [26]:
query = "What is the average property size in inclusive housing areas?"

In [27]:
input = alpaca_prompt1.format(
    query,
    "",
    "",)

In [28]:
input

'Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\nWhat is the average property size in inclusive housing areas?\n\n### sql:\n\n\n### Explanation:\n'